# Section 2.2: Roles and Structure


| **Aspect** | **Details** |
|-------------|-------------|
| **Goal** | Master role prompting personas and structured input patterns. |
| **Time** | ~20 minutes |
| **Prerequisites** | Complete Section 2.1 and confirm your environment is configured. |
| **Next Steps** | Continue to Section 2.3: Patterns for Reasoning |


---

## 🔧 Quick Setup Check

Since you completed Section 1, setup is already done! We just need to import it.

In [ ]:
# Quick setup check - imports setup_utils
try:
    import importlib
    import setup_utils
    importlib.reload(setup_utils)
    from setup_utils import *
    print(f"✅ Setup loaded! Using {AVAILABLE_PROVIDERS} with {get_default_model()}")
    print("🚀 Ready to build test generation templates!")
except ImportError:
    print("❌ Setup not found!")
    print("💡 Please run 2.1-setup-and-foundations.ipynb first to set up your environment.")



### 🎭 Tactic 1: Role Prompting

**Transform AI into specialized domain experts**

You can use role prompting as a way to get AI models to emulate certain styles in writing, speak in a certain voice, or guide the complexity of their answers. Role prompting can also make AI models better at performing math or logic tasks.

Using the `system` parameter for role prompting is the most powerful way to transform any LLM from a general assistant into your virtual domain expert. The right role enhances accuracy in complex scenarios, tailors the communication tone, and improves focus by keeping the LLM within the bounds of your task's specific requirements.

In coding scenarios, role prompting helps with tasks like specific refactoring requirements (e.g., "Extract this into separate classes following SOLID principles"), detailed code review criteria (e.g., "Focus on security vulnerabilities and performance bottlenecks"), and precise testing specifications (e.g., "Generate unit tests with 90% coverage including edge cases").

*Reference: [Claude Documentation - System Prompts](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/system-prompts)*

In [ ]:
# Instead of asking for a generic response, adopt a specific persona
messages = [
    {
        "role": "system",
        "content": "You are a code reviewer. Analyze the provided code and give exactly 3 specific feedback points: 1 about code structure, 1 about naming conventions, and 1 about potential improvements. Format each point as a bullet with the category in brackets.",
    },
    {
        "role": "user",
        "content": "def calc(x, y): return x + y if x > 0 and y > 0 else 0",
    }
]
response = get_chat_completion(messages)

print("CODE REVIEWER PERSONA RESULT:")
print(response)
print("\n" + "="*50 + "\n")

#### Software Engineering Personas

The below cells show how different engineering personas provide specialized expertise for code reviews.

In [ ]:
# Security Engineer Persona
security_messages = [
    {
        "role": "system", 
        "content": "You are a security engineer. Review code for security vulnerabilities and provide specific recommendations."
    },
    {
        "role": "user",
        "content": """Review this login function:
        
def login(username, password):
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    result = database.execute(query)
    return result"""
    }
]

security_response = get_chat_completion(security_messages)
print("🔒 SECURITY ENGINEER ANALYSIS:")
print(security_response)
print("\n" + "="*50 + "\n")


---

### 🎯 Try It Yourself: Role Prompting

**Common Misconception:** Generic code review requests produce the same quality results regardless of expertise focus.

**The Reality:** Different personas catch different issues. A security engineer spots vulnerabilities a performance engineer might miss.

**Your Task:** Below is a generic code review prompt. Fix it by:
1. Adding an appropriate role in the `system` message
2. Choosing between: Security Engineer, Performance Engineer, or QA Engineer
3. Specifying what that persona should focus on

Compare the generic review with your specialized review!

<div style="margin-top:16px; color:#78350f; padding:12px; background:#fef3c7; border-radius:6px; border-left:4px solid #f59e0b;">
<strong>💡 Tip:</strong> If you see long AI responses and the output shows "Output is truncated. View as a scrollable element" - <strong>click that link</strong> to see the full response in a scrollable view!
</div>

In [ ]:
# Code to review
code_to_review = """
def get_user_data(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    results = db.execute(query).fetchall()
    return results
"""

# ❌ BAD: Generic review (no specific expertise)
bad_messages = [
    {
        "role": "user",
        "content": f"Review this code:\n\n{code_to_review}"
    }
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("GENERIC REVIEW (No Role):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Add a role and specific focus
# TODO: Uncomment and complete this section
# Choose a role: Security Engineer, Performance Engineer, or QA Engineer
good_messages = [
    {
        "role": "system",
        "content": "You are a Security Engineer. Review code specifically for security vulnerabilities including SQL injection, authentication issues, and data exposure risks. Provide severity ratings."
    },
    {
        "role": "user",
        "content": f"Review this code:\n\n{code_to_review}"
    }
]

good_response = get_chat_completion(good_messages)
print("=" * 70)
print("SECURITY ENGINEER REVIEW (With Role):")
print("=" * 70)
print(good_response)

print("\n💡 Notice the difference? The security engineer immediately spots SQL injection!")

---
### 📋 Tactic 2: Structured Inputs

**Organize complex scenarios with delimiters**

When your prompts involve multiple components like context, instructions, and examples, it can be a game-changer to use delimiters that clearly separate these parts. Delimiters help AI models parse your prompts more accurately, leading to higher-quality outputs.

<div style="margin:16px 0; padding:12px; background:#dbeafe; border-radius:6px; border-left:4px solid #3b82f6; color:#1f2937;">
<style>
code {
  font-family: Consolas,"courier new";
  color:rgb(238, 13, 13);
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 110%;
}
</style>
<strong style="color:#1e40af;">📌 Building on Previous Tactics:</strong><br><br>
While <em>Tactics 0</em> and <em>1</em> used basic structure (separating system/user messages and organizing information in JSON format), Tactic 2 takes structure further by using explicit delimiters <em>within</em> your prompt content to organize complex, multi-part information.
<br><br>
<strong>Note:</strong> JSON format is required to pass messages to the <code>get_chat_completion()</code> function - that's the API's message structure. Tactic 2 adds delimiters <em>inside</em> the message content itself for better organization.
</div>

**Understanding Delimiters**

* Simple Delimiters
  - Use `###` or `#` to create visual section headers within your prompts
  - These act like markdown headers, making sections distinct and easy to identify
  - Example: `### CODE ###` followed by code, then `### REQUIREMENTS ###` followed by requirements
  - The LLM recognizes these as visual separators that indicate different content sections

* XML Delimiters
  - More powerful and precise than simple delimiters
  - LLMs are extensively trained on XML/HTML during their training, making them particularly good at parsing tagged content
  - You can create your own tag names that are descriptive (e.g., `<user_input>`, `<system_logs>`, `<test_cases>`)
  - Tag names should be meaningful and describe their content - while you have flexibility, clear names improve results
  - Common effective tags: `<instructions>`, `<example>`, `<context>`, `<requirements>`, `<code>`, `<output>`
  - LLMs have seen standard HTML/XML tags during training (like `<document>`, `<source>`, `<content>`), which is why the Claude documentation recommends certain tags

**Why This Works:**
- **Clarity:** Clearly separate different parts of your prompt and ensure your prompt is well structured
- **Accuracy:** Reduce errors caused by AI models misinterpreting parts of your prompt  
- **Flexibility:** Easily find, add, remove, or modify parts of your prompt without rewriting everything
- **Parseability:** Having the AI use delimiters in its output makes it easier to extract specific parts of its response

In multi-file refactoring, separating different files being modified becomes essential using delimiters like `<original_file>` and `<refactored_file>`. You can distinguish between `<requirements>` and `<existing_code>`, organize `<test_cases>`, `<edge_cases>`, and `<error_cases>`, and structure pull request reviews with `<pr_description>`, `<code_changes>`, and `<test_changes>`.

Let's start with a simple example showing how delimiters clarify different sections of your prompt by using `###` as delimiters:

In [ ]:
# Using delimiters to refactor code
function_code = "def process_data(items): return [x.upper() for x in items if len(x) > 3]"
requirements = "Follow PEP 8 style guide, add type hints, improve readability"

delimiter_messages = [
    {
        "role": "system",
        "content": "You are a Python code reviewer. Provide only the refactored code without explanations."
    },
    {
        "role": "user",
        "content": f"""Refactor this function based on the requirements:

### CODE ###
{function_code}
###

### REQUIREMENTS ###
{requirements}
###

Return only the improved function code."""
    }
]

delimiter_response = get_chat_completion(delimiter_messages)
print("🔧 REFACTORED CODE:")
print(delimiter_response)
print("\n" + "="*70 + "\n")

#### Multi-File Scenarios with XML Delimiters

One of the most powerful techniques for complex software development tasks is using XML tags and delimiters to structure your prompts. This approach dramatically improves AI accuracy and reduces misinterpretation.

**Key Benefits:**
- **Clarity**: Clearly separate different parts of your prompt (instructions, context, examples)
- **Accuracy**: Reduce errors caused by AI misinterpreting parts of your prompt
- **Flexibility**: Easily modify specific sections without rewriting everything
- **Parseability**: Structure AI outputs for easier post-processing

**Best Practices:**
- Use tags like `<instructions>`, `<example>`, and `<formatting>` to clearly separate different parts
- Be consistent with tag names throughout your prompts
- Nest tags hierarchically: `<outer><inner></inner></outer>` for structured content
- Choose meaningful tag names that describe their content

**Reference**: Learn more about XML tagging best practices in the [Claude Documentation on XML Tags](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags).

In coding scenarios, delimiters become essential for:

- **Multi-file refactoring** - Separate different files being modified: `<original_file>`, `<refactored_file>`
- **Code vs. requirements** - Distinguish between `<requirements>` and `<existing_code>`
- **Test scenarios** - Organize `<test_cases>`, `<edge_cases>`, `<error_cases>`
- **Pull request reviews** - Structure `<pr_description>`, `<code_changes>`, `<test_changes>`

The below cell demonstrates multi-file refactoring using XML delimiters to organize complex codebases.

In [ ]:
# Multi-file analysis with XML delimiters
multifile_messages = [
    {
        "role": "system",
        "content": "You are a software architect. Analyze the provided files and identify architectural concerns."
    },
    {
        "role": "user",
        "content": """
<user_model>
class User:
    def __init__(self, email, password):
        self.email = email
        self.password = password
    
    def save(self):
        # Save to database
        pass
</user_model>

<user_controller>
from flask import Flask, request
app = Flask(__name__)

@app.route('/register', methods=['POST'])
def register():
    email = request.form['email']
    password = request.form['password']
    user = User(email, password)
    user.save()
    return "User registered"
</user_controller>

<requirements>
- Follow separation of concerns
- Add input validation
- Implement proper error handling
- Use dependency injection
</requirements>

Provide architectural recommendations for improving this code structure.
"""
    }
]

multifile_response = get_chat_completion(multifile_messages)
print("🏗️ ARCHITECTURAL ANALYSIS:")
print(multifile_response)
print("\n" + "="*70 + "\n")

---

### 🎯 Try It Yourself: Structured Inputs

**Common Misconception:** AI can parse messy, unorganized prompts just as well as structured ones.

**The Reality:** Delimiters dramatically reduce misinterpretation and improve accuracy, especially with complex multi-part inputs.

**Your Task:** Below is a messy prompt with requirements, code, and context all jumbled together. Reorganize it using XML tags:
- `<requirements>` for what needs to be done
- `<current_code>` for the existing implementation
- `<context>` for background information

Run both versions and see how structure improves the analysis!

In [ ]:
# ❌ BAD: Messy, unstructured prompt
bad_messages = [
    {
        "role": "user",
        "content": """
I need to refactor this function: def send_email(to, subject, body): smtp.send(to, subject, body)

The requirements are: add error handling, implement retry logic with exponential backoff, add logging, and validate email format. This is for a high-traffic notification service that sends 10k emails per hour. The current implementation fails silently when SMTP server is down and doesn't validate email addresses. We need 99.9% delivery rate.

Please refactor this code following best practices.
"""
    }
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("MESSY PROMPT RESULT:")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Restructure with XML tags
# TODO: Uncomment and complete this section
# good_messages = [
#     {
#         "role": "user",
#         "content": """
# <current_code>
# def send_email(to, subject, body):
#     smtp.send(to, subject, body)
# </current_code>
# 
# <context>
# This is for a high-traffic notification service that sends 10k emails per hour.
# Current implementation fails silently when SMTP server is down.
# Email addresses are not validated.
# We need 99.9% delivery rate.
# </context>
# 
# <requirements>
# 1. Add error handling
# 2. Implement retry logic with exponential backoff
# 3. Add logging for monitoring
# 4. Validate email format before sending
# 5. Follow Python best practices
# </requirements>
# 
# Please refactor this code addressing all requirements.
# """
#     }
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("STRUCTURED PROMPT RESULT:")
# print("=" * 70)
# print(good_response)
# 
# print("\n💡 Notice how the structured version produces more organized, complete refactoring!")

<div style="margin:20px 0; padding:16px 24px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius:10px; color:#fff; text-align:center; box-shadow:0 4px 15px rgba(102,126,234,0.3);">
  <strong style="font-size:1.05em;">✨ Great work! Learning sticks better with breaks.</strong><br>
  <span style="font-size:0.92em; opacity:0.95; margin-top:4px; display:block;">Step away for 5 minutes—stretch, breathe, and return ready for the next tactics.</span>
</div>

---

<div style="margin:24px 0; padding:20px 24px; background:linear-gradient(135deg, #f8fafc 0%, #e2e8f0 100%); border-radius:12px; border-left:5px solid #8b5cf6; box-shadow:0 2px 8px rgba(0,0,0,0.1);">
  <div style="color:#1e293b; font-size:0.85em; font-weight:600; text-transform:uppercase; letter-spacing:1px; margin-bottom:8px;">⏭️ Next Section</div>
  <div style="color:#0f172a; font-size:1.15em; font-weight:700; margin-bottom:6px;">Section 2.3: Patterns for Reasoning</div>
  <div style="color:#475569; font-size:0.95em; line-height:1.5; margin-bottom:12px;">Learn few-shot exemplars, chain-of-thought reasoning, and reference citations for reliable responses.</div>
  <a href="./2.3-patterns-for-reasoning.ipynb" style="display:inline-block; padding:8px 16px; background:#8b5cf6; color:#fff; text-decoration:none; border-radius:6px; font-weight:600; font-size:0.9em; transition:all 0.2s;">Continue to Section 2.3 →</a>
</div>